In [68]:
import pandas as pd

## Helper Functions For Plotting

In [69]:
def calculate_speedup_and_efficiency(df: pd.DataFrame, num_threads_column: str = 'num_threads', duration_column: str = 'save_duration_seconds') -> pd.DataFrame:
    """
    Calculates speedup and efficiency for a DataFrame of benchmark results.

    Assumes the DataFrame contains results for different numbers of threads,
    including a run with 1 thread, which serves as the baseline.

    Args:
        df (pd.DataFrame): The input DataFrame containing benchmark results.
                          Must have columns for thread count and duration.
        num_threads_column (str): The name of the column containing the number of threads.
        duration_column (str): The name of the column containing the duration (runtime) in seconds.

    Returns:
        pd.DataFrame: A new DataFrame with 'speedup' and 'efficiency' columns added,
                      or an empty DataFrame if a single-thread baseline is not found.
    """
    if df.empty:
        print("Warning: Input DataFrame is empty.")
        return pd.DataFrame()

    # Get the baseline runtime (1 thread)
    baseline_row = df[df[num_threads_column] == 1]
    if baseline_row.empty:
        print(f"Error: Baseline (1 {num_threads_column}) not found in the DataFrame. Cannot calculate speedup.")
        return pd.DataFrame()

    baseline_runtime = baseline_row[duration_column].iloc[0]

    if baseline_runtime == 0:
        print("Warning: Baseline runtime is zero. Speedup and efficiency calculations will result in division by zero.")
        return pd.DataFrame()

    # Calculate Speedup
    # Speedup = Runtime (1 thread) / Runtime (P threads)
    df['speedup'] = baseline_runtime / df[duration_column]

    # Calculate Efficiency
    # Efficiency = Speedup / Number of threads
    df['efficiency'] = df['speedup'] / df[num_threads_column]

    return df

In [70]:
def extract_efficiency_and_speedup_at_threads(dfs: list[pd.DataFrame], threads = 2):
    extracted_data = []
    for i, df in enumerate(dfs):
        df_at_threads = df[df['num_threads'] == threads]
        efficiency_val = df_at_threads['efficiency'].iloc[0]
        value_size_val = df_at_threads['value_size'].iloc[0]
        speedup_val = df_at_threads['speedup'].iloc[0]

        extracted_data.append({'value_size': value_size_val, f'efficiency_at_{threads}_threads': efficiency_val, f'speedup_at_{threads}_threads': speedup_val })

    result_df = pd.DataFrame(extracted_data)
    result_df = result_df.sort_values(by='value_size').reset_index(drop=True)
    
    return result_df

In [71]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def plot_workload_performance(df_metrics: pd.DataFrame, threads: int = 2):
    """
    Generates an interactive Plotly graph showing efficiency and speedup
    vs. value_size for a specific number of threads.

    Args:
        df_metrics (pd.DataFrame): DataFrame containing 'value_size',
                                   'efficiency_at_{threads}_threads',
                                   and 'speedup_at_{threads}_threads' columns.
                                   This DataFrame is typically generated by
                                   extract_efficiency_and_speedup_at_threads.
        threads (int): The number of threads for which the data is being plotted.
    """
    if df_metrics.empty:
        print("Error: Input DataFrame is empty. Cannot generate plot.")
        return

    # Check for required columns
    required_efficiency_col = f'efficiency_at_{threads}_threads'
    required_speedup_col = f'speedup_at_{threads}_threads'
    required_cols = ['value_size', required_efficiency_col, required_speedup_col]

    if not all(col in df_metrics.columns for col in required_cols):
        print(f"Error: DataFrame must contain '{required_cols}' columns for plotting.")
        return

    # Define fixed y-axis ranges for consistent comparison
    # Adjust these ranges based on your expected min/max values across all datasets
    # For speedup, it typically starts at 1.0 (baseline) and goes up.
    # For efficiency, it's typically between 0.0 and 1.0 (or slightly above 1.0 for superlinear).
    speedup_y_range = [0.0, 5.0]  # Example: from 0x to 5x speedup
    efficiency_y_range = [0.0, 1.5] # Example: from 0% to 150% efficiency (to accommodate superlinear)

    # Create subplots: one for Speedup, one for Efficiency
    fig = make_subplots(rows=1, cols=2,
                        subplot_titles=(f'Speedup at {threads} Threads vs. Value Size',
                                        f'Efficiency at {threads} Threads vs. Value Size'))

    # Plot Speedup
    fig.add_trace(
        go.Scatter(
            x=df_metrics['value_size'],
            y=df_metrics[required_speedup_col],
            mode='lines+markers',
            name=f'Speedup at {threads} Threads',
            marker=dict(symbol='circle', size=8),
            line=dict(width=2)
        ),
        row=1, col=1
    )
    fig.add_trace(
        go.Scatter(
            x=df_metrics['value_size'],
            y=[1] * len(df_metrics), # Baseline for speedup
            mode='lines',
            name='Ideal (1x Speedup)',
            line=dict(dash='dash', color='gray'),
            showlegend=True
        ),
        row=1, col=1
    )

    # Plot Efficiency
    fig.add_trace(
        go.Scatter(
            x=df_metrics['value_size'],
            y=df_metrics[required_efficiency_col],
            mode='lines+markers',
            name=f'Efficiency at {threads} Threads',
            marker=dict(symbol='diamond', size=8),
            line=dict(width=2)
        ),
        row=1, col=2
    )
    fig.add_trace(
        go.Scatter(
            x=df_metrics['value_size'],
            y=[1] * len(df_metrics), # Ideal efficiency
            mode='lines',
            name='Ideal Efficiency (100%)',
            line=dict(dash='dash', color='gray'),
            showlegend=True
        ),
        row=1, col=2
    )
    fig.add_trace(
        go.Scatter(
            x=df_metrics['value_size'],
            y=[0.5] * len(df_metrics), # 50% efficiency reference
            mode='lines',
            name='50% Efficiency',
            line=dict(dash='dot', color='orange'),
            showlegend=True
        ),
        row=1, col=2
    )

    # Update layout for a beautiful graph
    fig.update_layout(
        title_text=f'Performance Scaling with Workload (at {threads} Threads)',
        height=600, width=1200,
        showlegend=True,
        hovermode="x unified", # Shows hover info for all traces at x-position
        template="plotly_white", # Clean white background
        title_x=0.5, # Center the main title
    )

    # Update x-axes to be consistent and descriptive
    fig.update_xaxes(
        title_text='Value Size (Bytes)',
        type='log', # Use log scale if value_size spans orders of magnitude
        tickmode='array',
        tickvals=df_metrics['value_size'].tolist(), # Set specific tick values
        ticktext=[f'{s}B' for s in df_metrics['value_size'].tolist()], # Add 'B' for bytes
        row=1, col=1
    )
    fig.update_xaxes(
        title_text='Value Size (Bytes)',
        type='log', # Use log scale if value_size spans orders of magnitude
        tickmode='array',
        tickvals=df_metrics['value_size'].tolist(),
        ticktext=[f'{s}B' for s in df_metrics['value_size'].tolist()],
        row=1, col=2
    )

    # Update y-axes for clarity and consistent ranges
    fig.update_yaxes(title_text='Speedup (X times faster)', range=speedup_y_range, row=1, col=1)
    fig.update_yaxes(title_text='Efficiency (Fraction of Ideal)', range=efficiency_y_range, row=1, col=2)

    fig.show()
    # You can also save it as an HTML file for interactive viewing later:
    # fig.write_html(f"performance_scaling_at_{threads}_threads.html")

In [72]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def plot_single_df_performance(df: pd.DataFrame, workload_description: str = "Performance Metrics"):
    """
    Generates an interactive Plotly graph showing speedup and efficiency
    vs. number of threads for a single DataFrame, with consistent y-axis ranges.

    Args:
        df (pd.DataFrame): The input DataFrame containing 'num_threads', 'speedup',
                           and 'efficiency' columns.
        workload_description (str): A descriptive string for the plot title,
                                    e.g., "Value Size 50 Bytes, No Compression".
    """
    if df.empty or 'num_threads' not in df.columns or 'speedup' not in df.columns or 'efficiency' not in df.columns:
        print("Error: DataFrame must contain 'num_threads', 'speedup', and 'efficiency' columns for plotting.")
        return

    # Define fixed y-axis ranges for consistent comparison across all your plots
    # IMPORTANT: Adjust these ranges based on the MIN and MAX values you expect
    # across ALL your benchmark datasets for both speedup and efficiency.
    speedup_y_range = [0.0, 5.0]  # Example: from 0x to 5x speedup
    efficiency_y_range = [0.0, 1.5] # Example: from 0% to 150% efficiency (to accommodate superlinear)

    # Create subplots: one for Speedup, one for Efficiency
    fig = make_subplots(rows=1, cols=2,
                        subplot_titles=(f'Speedup', f'Efficiency'))

    # Plot Speedup
    fig.add_trace(
        go.Scatter(
            x=df['num_threads'],
            y=df['speedup'],
            mode='lines+markers',
            name='Speedup',
            marker=dict(symbol='circle', size=8),
            line=dict(width=2, color='royalblue')
        ),
        row=1, col=1
    )
    fig.add_trace(
        go.Scatter(
            x=df['num_threads'],
            y=[1] * len(df), # Baseline for speedup
            mode='lines',
            name='Ideal Speedup',
            line=dict(dash='dash', color='gray'),
            showlegend=True
        ),
        row=1, col=1
    )

    # Plot Efficiency
    fig.add_trace(
        go.Scatter(
            x=df['num_threads'],
            y=df['efficiency'],
            mode='lines+markers',
            name='Efficiency',
            marker=dict(symbol='diamond', size=8),
            line=dict(width=2, color='firebrick')
        ),
        row=1, col=2
    )
    fig.add_trace(
        go.Scatter(
            x=df['num_threads'],
            y=[1] * len(df), # Ideal efficiency
            mode='lines',
            name='Ideal Efficiency (100%)',
            line=dict(dash='dash', color='gray'),
            showlegend=True
        ),
        row=1, col=2
    )
    fig.add_trace(
        go.Scatter(
            x=df['num_threads'],
            y=[0.5] * len(df), # 50% efficiency reference
            mode='lines',
            name='50% Efficiency',
            line=dict(dash='dot', color='orange'),
            showlegend=True
        ),
        row=1, col=2
    )

    # Update layout for a beautiful graph
    fig.update_layout(
        title_text=f'{workload_description} vs. Number of Threads',
        height=600, width=1200,
        showlegend=True,
        hovermode="x unified", # Shows hover info for all traces at x-position
        template="plotly_white", # Clean white background
        title_x=0.5, # Center the main title
    )

    # Update x-axes
    fig.update_xaxes(
        title_text='Number of Threads',
        tickmode='array',
        tickvals=df['num_threads'].unique(), # Ensure ticks match thread counts
        row=1, col=1
    )
    fig.update_xaxes(
        title_text='Number of Threads',
        tickmode='array',
        tickvals=df['num_threads'].unique(),
        row=1, col=2
    )

    # Update y-axes for clarity and consistent ranges
    # The key change for consistent axes is setting the 'range' property
    fig.update_yaxes(title_text='Speedup (X times faster)', range=speedup_y_range, row=1, col=1)
    fig.update_yaxes(title_text='Efficiency (Fraction of Ideal)', range=efficiency_y_range, row=1, col=2)

    fig.show()
    # You can also save it as an HTML file for interactive viewing later:
    # fig.write_html(f"performance_metrics_{workload_description.replace(' ', '_').replace('.', '')}.html")

In [73]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px # Import plotly.express for its color sequences

def format_size(size: int) -> str:
    """
    Converts an integer byte value into a human-readable string
    using KB (Kilobytes) and MB (Megabytes) units.
    """
    if size >= 1000000:
        return f'{size / 1000000:.0f}MB'
    elif size >= 1000:
        return f'{size / 1000:.0f}KB'
    else:
        return f'{size}B'
    
    
def plot_combined_workload_performance(dfs: list[pd.DataFrame], workload_type_description: str = "Overall Performance"):
    """
    Generates an interactive Plotly graph showing speedup and efficiency
    vs. value_size for multiple thread counts on the same chart, with distinct colors.

    Args:
        dfs (list[pd.DataFrame]): A list of DataFrames, where each DataFrame
                                   contains 'num_threads', 'value_size', 'speedup',
                                   and 'efficiency' columns for a specific workload.
                                   It's assumed 'speedup' and 'efficiency' have
                                   already been calculated for each df.
        workload_type_description (str): A general description for the plot title,
                                         e.g., "with Compression ON".
    """
    if not dfs:
        print("Error: Input list of DataFrames is empty. Cannot generate plot.")
        return

    # Combine all DataFrames into a single one for easier processing
    combined_df = pd.concat(dfs, ignore_index=True)

    # Ensure required columns exist
    required_cols = ['num_threads', 'value_size', 'speedup', 'efficiency']
    if not all(col in combined_df.columns for col in required_cols):
        print(f"Error: Combined DataFrame must contain '{required_cols}' columns for plotting.")
        return

    # Ensure speedup/efficiency are numeric
    combined_df['speedup'] = pd.to_numeric(combined_df['speedup'], errors='coerce')
    combined_df['efficiency'] = pd.to_numeric(combined_df['efficiency'], errors='coerce')
    combined_df.dropna(subset=['speedup', 'efficiency'], inplace=True)

    # Define fixed y-axis ranges for consistent comparison
    speedup_y_range = [0.0, max(5.0, combined_df['speedup'].max() * 1.1)]
    efficiency_y_range = [0.0, max(1.1, combined_df['efficiency'].max() * 1.1)]

    # Get unique thread counts for different lines
    unique_threads = sorted(combined_df['num_threads'].unique())
    # Get unique value sizes for x-axis ticks
    unique_value_sizes = sorted(combined_df['value_size'].unique())

    # --- Color and Marker Definition ---
    # Using Plotly Express's default qualitative color sequence for distinctness
    colors = px.colors.qualitative.Plotly
    # Define a list of distinct marker symbols
    markers = ['circle', 'square', 'diamond', 'cross', 'x', 'triangle-up', 'triangle-down', 'pentagon', 'hexagram']
    
    # Map thread count to a color and marker
    color_map = {thread: colors[i % len(colors)] for i, thread in enumerate(unique_threads)}
    marker_map = {thread: markers[i % len(markers)] for i, thread in enumerate(unique_threads)}
    # --- End Color and Marker Definition ---


    # Create subplots: one for Speedup, one for Efficiency
    fig = make_subplots(rows=1, cols=2,
                        subplot_titles=(f'Speedup vs. Value Size',
                                        f'Efficiency vs. Value Size'))

    # Plot Speedup for each number of threads
    for threads_count in unique_threads:
        df_subset = combined_df[combined_df['num_threads'] == threads_count]
        fig.add_trace(
            go.Scatter(
                x=df_subset['value_size'],
                y=df_subset['speedup'],
                mode='lines+markers',
                name=f'{threads_count} Threads',
                legendgroup=str(threads_count), # Group traces for consistent legend
                showlegend=True, # Show legend for each thread group
                marker=dict(symbol=marker_map[threads_count], size=8), # Apply distinct marker
                line=dict(width=2, color=color_map[threads_count]) # Apply distinct color
            ),
            row=1, col=1
        )
    # Add ideal speedup line
    fig.add_trace(
        go.Scatter(
            x=unique_value_sizes,
            y=[1] * len(unique_value_sizes),
            mode='lines',
            name='1x Speedup',
            line=dict(dash='dash', color='gray'), # Keep ideal lines distinct and subtle
            showlegend=True,
            legendgroup='ideal_speedup'
        ),
        row=1, col=1
    )


    # Plot Efficiency for each number of threads
    for threads_count in unique_threads:
        df_subset = combined_df[combined_df['num_threads'] == threads_count]
        fig.add_trace(
            go.Scatter(
                x=df_subset['value_size'],
                y=df_subset['efficiency'],
                mode='lines+markers',
                name=f'{threads_count} Threads',
                legendgroup=str(threads_count), # Group traces for consistent legend
                showlegend=False, # Hide legend for efficiency to avoid duplication
                marker=dict(symbol=marker_map[threads_count], size=8), # Apply distinct marker
                line=dict(width=2, color=color_map[threads_count]) # Apply distinct color
            ),
            row=1, col=2
        )
    # Add ideal efficiency lines
    fig.add_trace(
        go.Scatter(
            x=unique_value_sizes,
            y=[1] * len(unique_value_sizes),
            mode='lines',
            name='Ideal Efficiency (100%)',
            line=dict(dash='dash', color='gray'),
            showlegend=True,
            legendgroup='ideal_efficiency_1'
        ),
        row=1, col=2
    )
    fig.add_trace(
        go.Scatter(
            x=unique_value_sizes,
            y=[0.5] * len(unique_value_sizes),
            mode='lines',
            name='50% Efficiency',
            line=dict(dash='dot', color='orange'),
            showlegend=True,
            legendgroup='ideal_efficiency_05'
        ),
        row=1, col=2
    )


    # Update layout for a beautiful graph
    fig.update_layout(
        title_text=workload_type_description,
        height=600, width=1200,
        showlegend=True,
        hovermode="x unified",
        template="plotly_white",
        title_x=0.5,
    )

    # Update x-axes to be consistent and descriptive
    fig.update_xaxes(
        title_text='Value Size (Bytes)',
        type='log',
        tickmode='array',
        tickvals=unique_value_sizes,
        ticktext=[f'{format_size(s)}' for s in unique_value_sizes],
        row=1, col=1
    )
    fig.update_xaxes(
        title_text='Value Size (Bytes)',
        type='log',
        tickmode='array',
        tickvals=unique_value_sizes,
        ticktext=[f'{s}B' for s in unique_value_sizes],
        row=1, col=2
    )

    # Update y-axes for clarity and consistent ranges
    fig.update_yaxes(title_text='Speedup (X times faster)', range=speedup_y_range, row=1, col=1)
    fig.update_yaxes(title_text='Efficiency (Fraction of Ideal)', range=efficiency_y_range, row=1, col=2)

    fig.show()
    # fig.write_html(f"combined_performance_scaling_{workload_type_description.replace(' ', '_').replace('.', '')}.html")

In [74]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px # For distinct colors and markers

def plot_performance_by_workload_on_threads(dfs: list[pd.DataFrame], workload_names: list[str], title: str):
    """
    Generates an interactive Plotly graph showing speedup and efficiency
    vs. number of threads, with each line representing a different workload.

    Args:
        dfs (list[pd.DataFrame]): A list of DataFrames, where each DataFrame
                                   contains 'num_threads', 'value_size', 'speedup',
                                   and 'efficiency' columns for a specific workload.
                                   It's assumed 'speedup' and 'efficiency' have
                                   already been calculated for each df.
        workload_names (list[str]): A list of descriptive names for each workload,
                                     used for legend labels (e.g., ["50 Bytes", "500 Bytes"]).
                                     Length must match dfs.
    """
    if not dfs or not workload_names or len(dfs) != len(workload_names):
        print("Error: Input lists 'dfs' and 'workload_names' must not be empty and must have matching lengths.")
        return

    # Define fixed y-axis ranges for consistent comparison across all your plots
    # Adjust these ranges based on the MIN and MAX values you expect across ALL your datasets.
    speedup_y_range = [0.0, 12]  # Example: from 0x to 5x speedup
    efficiency_y_range = [0.0, 2] # Example: from 0% to 150% efficiency (to accommodate superlinear)

    # Get all unique thread counts across all DFs for x-axis ticks
    all_num_threads = sorted(pd.concat(df['num_threads'] for df in dfs).unique())

    # --- Color and Marker Definition ---
    colors = px.colors.qualitative.Plotly # A good default palette
    markers = ['circle', 'square', 'diamond', 'cross', 'x', 'triangle-up', 'triangle-down', 'pentagon', 'hexagram']
    # --- End Color and Marker Definition ---

    # Create subplots: one for Speedup, one for Efficiency
    fig = make_subplots(rows=1, cols=2,
                        subplot_titles=(f'Speedup vs. Number of Threads',
                                        f'Efficiency vs. Number of Threads'))

    # Plot Speedup for each workload
    for i, df in enumerate(dfs):
        workload_label = workload_names[i]
        color = colors[i % len(colors)]
        marker = markers[i % len(markers)]

        fig.add_trace(
            go.Scatter(
                x=df['num_threads'],
                y=df['speedup'],
                mode='lines+markers',
                name=f'Speedup: {workload_label}',
                marker=dict(symbol=marker, size=8),
                line=dict(width=2, color=color),
                legendgroup=workload_label, # Group traces for consistent legend
                showlegend=True
            ),
            row=1, col=1
        )

    # Add Ideal Speedup line (for reference, only once)
    fig.add_trace(
        go.Scatter(
            x=all_num_threads,
            y=[1] * len(all_num_threads),
            mode='lines',
            name='(1x Speedup)',
            line=dict(dash='dash', color='gray'),
            showlegend=True,
            legendgroup='ideal_speedup'
        ),
        row=1, col=1
    )


    # Plot Efficiency for each workload
    for i, df in enumerate(dfs):
        workload_label = workload_names[i]
        color = colors[i % len(colors)]
        marker = markers[i % len(markers)]

        fig.add_trace(
            go.Scatter(
                x=df['num_threads'],
                y=df['efficiency'],
                mode='lines+markers',
                name=f'Efficiency: {workload_label}',
                marker=dict(symbol=marker, size=8),
                line=dict(width=2, color=color),
                legendgroup=workload_label, # Group traces for consistent legend
                showlegend=False # Hide legend here to avoid duplication, as it's shown for Speedup
            ),
            row=1, col=2
        )

    # Add Ideal Efficiency lines (for reference, only once)
    fig.add_trace(
        go.Scatter(
            x=all_num_threads,
            y=[1] * len(all_num_threads),
            mode='lines',
            name='Ideal Efficiency (100%)',
            line=dict(dash='dash', color='gray'),
            showlegend=True,
            legendgroup='ideal_efficiency_1'
        ),
        row=1, col=2
    )
    fig.add_trace(
        go.Scatter(
            x=all_num_threads,
            y=[0.5] * len(all_num_threads),
            mode='lines',
            name='50% Efficiency',
            line=dict(dash='dot', color='orange'),
            showlegend=True,
            legendgroup='ideal_efficiency_05'
        ),
        row=1, col=2
    )


    # Update layout for a beautiful graph
    fig.update_layout(
        title_text=title,
        height=600, width=1200,
        showlegend=True,
        hovermode="x unified",
        template="plotly_white",
        title_x=0.5,
    )

    # Update x-axes for clarity
    fig.update_xaxes(
        title_text='Number of Threads',
        tickmode='array',
        tickvals=all_num_threads,
        row=1, col=1
    )
    fig.update_xaxes(
        title_text='Number of Threads',
        tickmode='array',
        tickvals=all_num_threads,
        row=1, col=2
    )

    # Update y-axes for clarity and consistent ranges
    fig.update_yaxes(title_text='Speedup (X times faster)', range=speedup_y_range, row=1, col=1)
    fig.update_yaxes(title_text='Efficiency (Fraction of Ideal)', range=efficiency_y_range, row=1, col=2)

    fig.show()
    # fig.write_html(output_filename)
    # You can also save it as an HTML file for interactive viewing later:
    # fig.write_html(f"performance_scaling_across_workloads.html")

In [75]:
import os
os.listdir('./')

['save_summary_50.0keys_5000B_comp-no_csum-yes.csv',
 'save_summary_2000.0keys_50B_comp-no_csum-yes.csv',
 'save_summary_10.0keys_20000B_comp-no_csum-yes.csv',
 'performance_improvement_vs_value_size.ipynb',
 'save_summary_500.0keys_500B_comp-no_csum-yes.csv',
 'save_summary_0.1keys_1000000B_comp-no_csum-yes.csv',
 'save_summary_5.0keys_66000B_comp-no_csum-yes.csv',
 'save_summary_5.0keys_64000B_comp-no_csum-yes.csv',
 'save_summary_0.5keys_272000B_comp-no_csum-yes.csv',
 'save_summary_1.0keys_100000B_comp-no_csum-yes.csv',
 'save_summary_1000.0keys_300B_comp-no_csum-yes.csv',
 'save_summary_0.5keys_262000B_comp-no_csum-yes.csv']

## Loading and Plotting The Speed up and efficiency of the program for differnet thread configs and workloads

In [ ]:
test_configs = [
    {"num_keys": 2000.0, "value_size": 50},
    {"num_keys": 1000.0, "value_size": 300},
    {"num_keys": 500.0, "value_size": 500},
    {"num_keys": 50.0, "value_size": 5000},
    {"num_keys": 10.0, "value_size": 20000},
    {"num_keys": 5.0, "value_size": 64000},
    {"num_keys": 5.0, "value_size": 66000},
    {"num_keys": 1.0, "value_size": 100000},
    {"num_keys": 0.5, "value_size": 262000},
    {"num_keys": 0.5, "value_size": 272000},
    {"num_keys": 0.1, "value_size": 1000000},

    ]


def load_df(test_config: dict, compression_on: bool):
    num_keys = test_config["num_keys"]
    value_size = test_config["value_size"]
    compression_str = "yes" if compression_on else "no"
    df = pd.read_csv(f"save_summary_{num_keys}keys_{value_size}B_comp-no_csum-yes.csv")
    
    ret_df = df[df["rdbcompression"] == compression_str].copy()
    return ret_df


dfs_compression_on = [load_df(config, compression_on=True) for config in test_configs]
dfs_compression_on = [calculate_speedup_and_efficiency(df) for df in dfs_compression_on]
dfs_compression_off = [load_df(config, compression_on=False) for config in test_configs]
dfs_compression_off = [calculate_speedup_and_efficiency(df) for df in dfs_compression_off]

print("\n" + "="*50) # Top border
print("                 COMPRESSION ON")
print("="*50 + "\n") # Bottom border

all_workload_names = [f"Value Size {format_size(config["value_size"])}" for config in test_configs]
plot_performance_by_workload_on_threads(dfs_compression_on, all_workload_names, "Speedup / Efficiency Vs. Num Threads for each workload Compression ON")
plot_combined_workload_performance(dfs_compression_on )

print("\n" + "="*50) # Top border
print("                 COMPRESSION OFF")
print("="*50 + "\n") # Bottom border

plot_performance_by_workload_on_threads(dfs_compression_off, all_workload_names, "Speedup / Efficiency Vs. Num Threads for each workload Compression OFF")
plot_combined_workload_performance(dfs_compression_off)
        


                 COMPRESSION ON




                 COMPRESSION OFF



In [77]:
import pandas as pd
import plotly.express as px
import numpy as np

def plot_raw_times_bar_chart(df: pd.DataFrame, workload_name: str):
    """
    Generates a bar chart showing raw save duration vs. number of threads,
    with separate bars for compression ON and OFF for each thread count,
    and displays the plot directly.
    
    Args:
        df (pd.DataFrame): A DataFrame containing 'num_threads', 'save_duration_seconds',
                           and 'rdbcompression' columns for a single workload.
        workload_name (str): The name of the workload for the plot title.
    """
    if df is None or df.empty:
        print("Error: Input DataFrame is empty or invalid. Cannot generate plot.")
        return

    # Ensure required columns are present and numeric where needed
    required_cols = ['num_threads', 'save_duration_seconds', 'rdbcompression']
    if not all(col in df.columns for col in required_cols):
        print(f"Error: DataFrame must contain '{required_cols}' columns for plotting.")
        return
    
    df['save_duration_seconds'] = pd.to_numeric(df['save_duration_seconds'], errors='coerce')
    df['num_threads'] = df['num_threads'].astype(str)
    df['save_duration_minutes'] = df['save_duration_seconds'] / 60


    fig = px.bar(
        df,
        x='num_threads',
        y='save_duration_minutes',
        color='rdbcompression',
        barmode='group',
        title=f'Raw Save Times for Workload: {workload_name}',
        labels={'num_threads': 'Number of Threads', 'save_duration_minutes': 'Save Duration (Minutes)', 'rdbcompression': 'Compression'},
        template="plotly_white",
    )

    fig.update_layout(
        title_x=0.5,
        xaxis_title="Number of Threads",
        yaxis_title="Save Duration (seconds)",
        legend_title="RDB Compression",
    )

    fig.show()


## Plotting the Raw Save Time for each workload for each thread configuration

In [ ]:
combined_dfs = []
for i in range(len(dfs_compression_on)):
    combined_df = pd.concat([dfs_compression_on[i], dfs_compression_off[i]], ignore_index=True)
    combined_dfs.append(combined_df)

for df in combined_dfs:
    if not df.empty:
        num_keys = f"{round(df['keys'].iloc[0] * 1e-6, 1)}"
        value_size = format_size(df['value_size'].iloc[0])
        workload_name = f"Num Keys: {num_keys} Million, Value Size: {format_size(df['value_size'].iloc[0])}"
        
        plot_raw_times_bar_chart(df, workload_name)
        

In [104]:
def print_raw_data(df):
    print(f"Num Keys: {df['keys'].iloc[0]}, Value Size: {df['value_size'].iloc[0]} Bytes ")
    return df

In [107]:
for df_comp_on, df_comp_off in zip(dfs_compression_on, dfs_compression_off):
    display(print_raw_data(df_comp_on))
    display(print_raw_data(df_comp_off))


Num Keys: 2000000000, Value Size: 50 Bytes 


,keys,value_size,num_threads,rdbcompression,rdbchecksum,valkey_data_throughput_mb_s,actual_throughput_mb_s,rdb_file_size_bytes,status,port,save_duration_seconds,cpu_utilization_percent,cpu_total_time_seconds,io_read_bytes,io_write_bytes,memory_rss_bytes,context_switches_voluntary,context_switches_involuntary,speedup,efficiency
0,2000000000,50,1,yes,yes,41.031133,42.896185,138000000112,ok,7000,3217.069333,99.963341,3215.89,0,138000007168,252282343424,707,14031,1.000000,1.000000
1,2000000000,50,2,yes,yes,136.101887,142.288337,138000000112,ok,7000,969.861643,195.934133,1900.29,0,138000007168,252283006976,40227,2097,3.317039,1.658520
2,2000000000,50,3,yes,yes,139.658651,146.006772,138000000112,ok,7000,945.161639,290.975626,2750.19,0,138000007168,252283203584,108766,2391,3.403724,1.134575
3,2000000000,50,4,yes,yes,265.472880,277.539829,138000000112,ok,7000,497.225932,366.089112,1820.29,1130496,138000007168,252282662912,103283,1125,6.470035,1.617509
4,2000000000,50,6,yes,yes,367.117945,383.805125,138000000112,ok,7000,359.557471,509.676518,1832.58,28811264,138000007168,252281053184,105029,1145,8.947302,1.491217
5,2000000000,50,8,yes,yes,394.326507,412.250439,138000000112,ok,7000,334.747976,642.351905,2150.26,5320704,138000007168,252283076608,158001,891,9.610422,1.201303
6,2000000000,50,10,yes,yes,417.287933,436.255567,138000000112,ok,7000,316.328342,758.398690,2399.03,3182592,138000007168,252283547648,163550,693,10.170032,1.017003


Num Keys: 2000000000, Value Size: 50 Bytes 


,keys,value_size,num_threads,rdbcompression,rdbchecksum,valkey_data_throughput_mb_s,actual_throughput_mb_s,rdb_file_size_bytes,status,port,save_duration_seconds,cpu_utilization_percent,cpu_total_time_seconds,io_read_bytes,io_write_bytes,memory_rss_bytes,context_switches_voluntary,context_switches_involuntary,speedup,efficiency
7,2000000000,50,1,no,yes,76.738964,80.227099,138000000112,ok,7000,1720.117040,99.947269,1719.21,770048,138000007168,252282642432,614,8476,1.000000,1.000000
8,2000000000,50,2,no,yes,203.599844,212.854383,138000000112,ok,7000,648.330555,190.219324,1233.25,45056,138000007168,252282761216,27132,1916,2.653148,1.326574
9,2000000000,50,3,no,yes,211.115170,220.711315,138000000112,ok,7000,625.251135,279.193415,1745.66,729088,138000007168,252282830848,237570,1833,2.751082,0.917027
10,2000000000,50,4,no,yes,296.585503,310.066662,138000000112,ok,7000,445.065584,357.718066,1592.08,774144,138000007168,252282892288,212248,943,3.864862,0.966215
11,2000000000,50,6,no,yes,366.710431,383.379088,138000000112,ok,7000,359.957036,501.520965,1805.26,8192,138000007168,252282908672,165736,741,4.778673,0.796446
12,2000000000,50,8,no,yes,466.714611,487.928912,138000000112,ok,7000,282.828086,580.773297,1642.59,24576,138000007168,252283039744,202221,609,6.081847,0.760231
13,2000000000,50,10,no,yes,445.305397,465.546551,138000000112,ok,7000,296.425781,550.876511,1632.94,0,138000007168,252283166720,202894,645,5.802859,0.580286


Num Keys: 1000000000, Value Size: 300 Bytes 


...

,keys,value_size,num_threads,rdbcompression,rdbchecksum,valkey_data_throughput_mb_s,actual_throughput_mb_s,rdb_file_size_bytes,status,port,save_duration_seconds,cpu_utilization_percent,cpu_total_time_seconds,io_read_bytes,io_write_bytes,memory_rss_bytes,context_switches_voluntary,context_switches_involuntary,speedup,efficiency
7,500000,272000,1,no,yes,634.912029,634.928368,136011500112,ok,7000,214.215503,99.427911,212.99,0,136011505664,150528581632,26040,1174,1.000000,1.000000
8,500000,272000,2,no,yes,696.518542,696.536466,136011500112,ok,7000,195.268312,149.507105,291.94,0,136011505664,150528598016,273719,408,1.097032,0.548516
9,500000,272000,3,no,yes,652.129670,652.146452,136011500112,ok,7000,208.559749,137.586472,286.95,0,136011505664,150528606208,180653,534,1.027118,0.342373
10,500000,272000,4,no,yes,660.210593,660.227583,136011500112,ok,7000,206.006995,135.509962,279.16,0,136011505664,150528614400,115472,509,1.039846,0.259961
11,500000,272000,6,no,yes,708.883892,708.902134,136011500112,ok,7000,191.862168,165.040354,316.65,0,136011505664,150528618496,78326,229,1.116507,0.186085
12,500000,272000,8,no,yes,666.210754,666.227899,136011500112,ok,7000,204.151613,205.264114,419.05,0,136011505664,150528610304,53285,211,1.049296,0.131162
13,500000,272000,10,no,yes,651.285397,651.302157,136011500112,ok,7000,208.830108,228.721808,477.64,0,136011505664,150528614400,51926,126,1.025788,0.102579


Num Keys: 100000, Value Size: 1000000 Bytes 


,keys,value_size,num_threads,rdbcompression,rdbchecksum,valkey_data_throughput_mb_s,actual_throughput_mb_s,rdb_file_size_bytes,status,port,save_duration_seconds,cpu_utilization_percent,cpu_total_time_seconds,io_read_bytes,io_write_bytes,memory_rss_bytes,context_switches_voluntary,context_switches_involuntary,speedup,efficiency
0,100000,1000000,1,yes,yes,182.684176,182.685455,100002300112,ok,7000,547.401544,99.952221,547.14,0,100002308096,102288707584,276,1716,1.000000,1.000000
1,100000,1000000,2,yes,yes,358.763156,358.765668,100002300112,ok,7000,278.739883,197.115675,549.44,0,100002308096,102290440192,8121,652,1.963844,0.981922
2,100000,1000000,3,yes,yes,494.428839,494.432300,100002300112,ok,7000,202.256811,289.102749,584.73,0,100002308096,102290624512,5283,722,2.706468,0.902156
3,100000,1000000,4,yes,yes,630.865263,630.869679,100002300112,ok,7000,158.514989,361.536789,573.09,0,100002308096,102291787776,19620,368,3.453311,0.863328
4,100000,1000000,6,yes,yes,752.791137,752.796407,100002300112,ok,7000,132.841096,429.483056,570.53,0,100002308096,102294122496,95193,6658,4.120724,0.686787
5,100000,1000000,8,yes,yes,753.048793,753.054065,100002300112,ok,7000,132.795645,434.080501,576.44,0,100002308096,102297387008,114906,6202,4.122135,0.515267
6,100000,1000000,10,yes,yes,754.191068,754.196349,100002300112,ok,7000,132.594516,434.226102,575.76,0,100002308096,102299766784,57557,5203,4.128387,0.412839


Num Keys: 100000, Value Size: 1000000 Bytes 


,keys,value_size,num_threads,rdbcompression,rdbchecksum,valkey_data_throughput_mb_s,actual_throughput_mb_s,rdb_file_size_bytes,status,port,save_duration_seconds,cpu_utilization_percent,cpu_total_time_seconds,io_read_bytes,io_write_bytes,memory_rss_bytes,context_switches_voluntary,context_switches_involuntary,speedup,efficiency
7,100000,1000000,1,no,yes,702.924785,702.929706,100002300112,ok,7000,142.265008,99.195158,141.12,0,100002308096,102289096704,44278,659,1.000000,1.000000
8,100000,1000000,2,no,yes,754.073016,754.078296,100002300112,ok,7000,132.615274,99.882914,132.46,0,100002308096,102289113088,194423,406,1.072765,0.536382
9,100000,1000000,3,no,yes,754.762724,754.768008,100002300112,ok,7000,132.494090,112.148399,148.59,0,100002308096,102289121280,182668,221,1.073746,0.357915
10,100000,1000000,4,no,yes,754.871748,754.877033,100002300112,ok,7000,132.474954,132.213671,175.15,0,100002308096,102289129472,124118,101,1.073901,0.268475
11,100000,1000000,6,no,yes,754.865990,754.871275,100002300112,ok,7000,132.475964,118.149734,156.52,0,100002308096,102289133568,89378,103,1.073893,0.178982
12,100000,1000000,8,no,yes,754.864506,754.869791,100002300112,ok,7000,132.476225,178.801895,236.87,0,100002308096,102289125376,67794,89,1.073891,0.134236
13,100000,1000000,10,no,yes,754.868104,754.873388,100002300112,ok,7000,132.475593,215.768046,285.84,0,100002308096,102289129472,51887,50,1.073896,0.107390


In [ ]:
# 2 Billion 50 Byte Keys, Compression OFF
df = dfs_compression_off[0]
print(f"Num Keys: {df['keys'].iloc[0]}, Value Size: {df['value_size'].iloc[0]} Bytes ")
dfs_compression_off[0]

Num Keys: 2000000000, Value Size: 50 Bytes 


,keys,value_size,num_threads,rdbcompression,rdbchecksum,valkey_data_throughput_mb_s,actual_throughput_mb_s,rdb_file_size_bytes,status,port,save_duration_seconds,cpu_utilization_percent,cpu_total_time_seconds,io_read_bytes,io_write_bytes,memory_rss_bytes,context_switches_voluntary,context_switches_involuntary,speedup,efficiency
7,2000000000,50,1,no,yes,76.738964,80.227099,138000000112,ok,7000,1720.117040,99.947269,1719.21,770048,138000007168,252282642432,614,8476,1.000000,1.000000
8,2000000000,50,2,no,yes,203.599844,212.854383,138000000112,ok,7000,648.330555,190.219324,1233.25,45056,138000007168,252282761216,27132,1916,2.653148,1.326574
9,2000000000,50,3,no,yes,211.115170,220.711315,138000000112,ok,7000,625.251135,279.193415,1745.66,729088,138000007168,252282830848,237570,1833,2.751082,0.917027
10,2000000000,50,4,no,yes,296.585503,310.066662,138000000112,ok,7000,445.065584,357.718066,1592.08,774144,138000007168,252282892288,212248,943,3.864862,0.966215
11,2000000000,50,6,no,yes,366.710431,383.379088,138000000112,ok,7000,359.957036,501.520965,1805.26,8192,138000007168,252282908672,165736,741,4.778673,0.796446
12,2000000000,50,8,no,yes,466.714611,487.928912,138000000112,ok,7000,282.828086,580.773297,1642.59,24576,138000007168,252283039744,202221,609,6.081847,0.760231
13,2000000000,50,10,no,yes,445.305397,465.546551,138000000112,ok,7000,296.425781,550.876511,1632.94,0,138000007168,252283166720,202894,645,5.802859,0.580286


In [81]:
# 1 Billion 300 Byte Keys, Compression ON
dfs_compression_on[1]

,keys,value_size,num_threads,rdbcompression,rdbchecksum,valkey_data_throughput_mb_s,actual_throughput_mb_s,rdb_file_size_bytes,status,port,save_duration_seconds,cpu_utilization_percent,cpu_total_time_seconds,io_read_bytes,io_write_bytes,memory_rss_bytes,context_switches_voluntary,context_switches_involuntary,speedup,efficiency
0,1000000000,300,1,yes,yes,127.331248,128.943036,320000000112,ok,7000,2481.716028,99.973969,2481.07,0,320000008192,392415690752,552,10397,1.000000,1.000000
1,1000000000,300,2,yes,yes,267.669962,271.058189,320000000112,ok,7000,1180.558318,191.710987,2263.26,77824,320000008192,392416325632,413802,2974,2.102155,1.051077
2,1000000000,300,3,yes,yes,447.469243,453.133411,320000000112,ok,7000,706.193789,265.840628,1877.35,0,320000008192,392416526336,947910,1389,3.514214,1.171405
3,1000000000,300,4,yes,yes,661.553057,669.927146,320000000112,ok,7000,477.663880,336.311801,1606.44,0,320000008192,392416575488,276035,1249,5.195528,1.298882
4,1000000000,300,6,yes,yes,709.953786,718.940543,320000000112,ok,7000,445.099394,359.059127,1598.17,0,320000008192,392416972800,855480,1268,5.575645,0.929274
5,1000000000,300,8,yes,yes,682.966162,691.611303,320000000112,ok,7000,462.687638,329.291270,1523.59,0,320000008192,392417304576,530388,1426,5.363696,0.670462
6,1000000000,300,10,yes,yes,672.239841,680.749206,320000000112,ok,7000,470.070324,330.841987,1555.19,0,320000008192,392417468416,486648,1234,5.279457,0.527946


In [82]:
# 1 Billion 300 Byte Keys, Compression OFF
dfs_compression_off[1]

,keys,value_size,num_threads,rdbcompression,rdbchecksum,valkey_data_throughput_mb_s,actual_throughput_mb_s,rdb_file_size_bytes,status,port,save_duration_seconds,cpu_utilization_percent,cpu_total_time_seconds,io_read_bytes,io_write_bytes,memory_rss_bytes,context_switches_voluntary,context_switches_involuntary,speedup,efficiency
7,1000000000,300,1,no,yes,247.651108,250.785932,320000000112,ok,7000,1275.988639,99.964840,1275.54,0,320000008192,392416583680,383,4482,1.000000,1.000000
8,1000000000,300,2,no,yes,516.313030,522.848638,320000000112,ok,7000,612.031813,181.301033,1109.62,5128192,320000008192,392416608256,248185,1586,2.084840,1.042420
9,1000000000,300,3,no,yes,641.473709,649.593630,320000000112,ok,7000,492.615668,247.426154,1218.86,135168,320000008192,392416698368,739851,1392,2.590232,0.863411
10,1000000000,300,4,no,yes,733.813308,743.102084,320000000112,ok,7000,430.627241,236.933455,1020.30,0,320000008192,392416747520,1106781,1068,2.963093,0.740773
11,1000000000,300,6,no,yes,699.083925,707.933089,320000000112,ok,7000,452.020120,220.530006,996.84,0,320000008192,392416878592,762653,1645,2.822858,0.470476
12,1000000000,300,8,no,yes,669.714587,678.191987,320000000112,ok,7000,471.842791,217.712344,1027.26,4096,320000008192,392416940032,619141,1248,2.704266,0.338033
13,1000000000,300,10,no,yes,671.434793,679.933968,320000000112,ok,7000,470.633937,219.642044,1033.71,0,320000008192,392417050624,492193,909,2.711213,0.271121


In [83]:
dfs_compression_on[2]

,keys,value_size,num_threads,rdbcompression,rdbchecksum,valkey_data_throughput_mb_s,actual_throughput_mb_s,rdb_file_size_bytes,status,port,save_duration_seconds,cpu_utilization_percent,cpu_total_time_seconds,io_read_bytes,io_write_bytes,memory_rss_bytes,context_switches_voluntary,context_switches_involuntary,speedup,efficiency
0,500000000,500,1,yes,yes,159.891516,161.130986,260000000112,ok,7000,1613.594053,99.976199,1613.21,0,260000006144,299404406784,359,6834,1.000000,1.000000
1,500000000,500,2,yes,yes,394.860996,397.921934,260000000112,ok,7000,653.394492,192.416682,1257.24,77824,260000006144,299405164544,104316,1462,2.469556,1.234778
2,500000000,500,3,yes,yes,491.125830,494.933008,260000000112,ok,7000,525.323622,275.443544,1446.97,0,260000006144,299405369344,450784,1081,3.071619,1.023873
3,500000000,500,4,yes,yes,717.743037,723.306937,260000000112,ok,7000,359.460122,326.567518,1173.88,0,260000006144,299405586432,681232,824,4.488938,1.122234
4,500000000,500,6,yes,yes,667.136885,672.308489,260000000112,ok,7000,386.727231,337.434216,1304.95,0,260000006144,299405967360,672323,1249,4.172435,0.695406
5,500000000,500,8,yes,yes,656.585897,661.675711,260000000112,ok,7000,392.941733,337.141589,1324.77,0,260000006144,299406233600,479240,1145,4.106446,0.513306
6,500000000,500,10,yes,yes,609.145678,613.867738,260000000112,ok,7000,423.544005,330.201817,1398.55,0,260000006144,299406602240,381633,913,3.809744,0.380974


In [84]:
dfs_compression_off[2]

,keys,value_size,num_threads,rdbcompression,rdbchecksum,valkey_data_throughput_mb_s,actual_throughput_mb_s,rdb_file_size_bytes,status,port,save_duration_seconds,cpu_utilization_percent,cpu_total_time_seconds,io_read_bytes,io_write_bytes,memory_rss_bytes,context_switches_voluntary,context_switches_involuntary,speedup,efficiency
7,500000000,500,1,no,yes,223.424160,225.156131,260000000112,ok,7000,1154.754256,99.871466,1153.27,471040,260000006144,299405590528,852,4806,1.000000,1.000000
8,500000000,500,2,no,yes,453.783953,457.301658,260000000112,ok,7000,568.552498,183.304093,1042.18,155648,260000006144,299405828096,321082,1487,2.031042,1.015521
9,500000000,500,3,no,yes,677.675915,682.929217,260000000112,ok,7000,380.712955,240.963694,917.38,0,260000006144,299405897728,975322,1161,3.033136,1.011045
10,500000000,500,4,no,yes,718.583657,724.154073,260000000112,ok,7000,359.039616,232.339264,834.19,0,260000006144,299405959168,981826,1385,3.216231,0.804058
11,500000000,500,6,no,yes,674.368654,679.596319,260000000112,ok,7000,382.580060,229.133740,876.62,0,260000006144,299405946880,666034,1441,3.018334,0.503056
12,500000000,500,8,no,yes,665.992248,671.154979,260000000112,ok,7000,387.391896,226.259250,876.51,0,260000006144,299406057472,502552,1035,2.980843,0.372605
13,500000000,500,10,no,yes,661.075681,666.200299,260000000112,ok,7000,390.273016,226.305679,883.21,0,260000006144,299406172160,407684,734,2.958837,0.295884


In [85]:
dfs_compression_on[3]

,keys,value_size,num_threads,rdbcompression,rdbchecksum,valkey_data_throughput_mb_s,actual_throughput_mb_s,rdb_file_size_bytes,status,port,save_duration_seconds,cpu_utilization_percent,cpu_total_time_seconds,io_read_bytes,io_write_bytes,memory_rss_bytes,context_switches_voluntary,context_switches_involuntary,speedup,efficiency
0,50000000,5000,1,yes,yes,174.761797,174.901160,251000000112,ok,7000,1435.096254,99.938244,1434.21,0,251000004608,260812271616,542,6580,1.000000,1.000000
1,50000000,5000,2,yes,yes,345.380532,345.655955,251000000112,ok,7000,726.155578,187.032923,1358.15,0,251000004608,260812959744,134319,1992,1.976293,0.988147
2,50000000,5000,3,yes,yes,450.604275,450.963609,251000000112,ok,7000,556.585931,257.460694,1432.99,0,251000004608,260813168640,663955,1583,2.578391,0.859464
3,50000000,5000,4,yes,yes,556.657749,557.101655,251000000112,ok,7000,450.546140,314.063283,1415.00,0,251000004608,260813389824,570410,1412,3.185237,0.796309
4,50000000,5000,6,yes,yes,536.406844,536.834600,251000000112,ok,7000,467.555556,313.740256,1466.91,0,251000004608,260813639680,590018,1435,3.069360,0.511560
5,50000000,5000,8,yes,yes,520.911340,521.326740,251000000112,ok,7000,481.463889,309.720009,1491.19,0,251000004608,260814028800,440329,1012,2.980693,0.372587
6,50000000,5000,10,yes,yes,503.171174,503.572427,251000000112,ok,7000,498.438728,310.212653,1546.22,0,251000004608,260814372864,356911,783,2.879183,0.287918


In [86]:
dfs_compression_off[3]

,keys,value_size,num_threads,rdbcompression,rdbchecksum,valkey_data_throughput_mb_s,actual_throughput_mb_s,rdb_file_size_bytes,status,port,save_duration_seconds,cpu_utilization_percent,cpu_total_time_seconds,io_read_bytes,io_write_bytes,memory_rss_bytes,context_switches_voluntary,context_switches_involuntary,speedup,efficiency
7,50000000,5000,1,no,yes,401.050363,401.370180,251000000112,ok,7000,625.357868,99.941175,624.99,0,251000004608,260813217792,2737,2865,1.000000,1.000000
8,50000000,5000,2,no,yes,663.197024,663.725890,251000000112,ok,7000,378.168162,138.845639,525.07,0,251000004608,260813365248,1726853,2028,1.653650,0.826825
9,50000000,5000,3,no,yes,643.273720,643.786698,251000000112,ok,7000,389.880687,119.333944,465.26,0,251000004608,260813434880,1219954,1447,1.603972,0.534657
10,50000000,5000,4,no,yes,558.267539,558.712728,251000000112,ok,7000,449.246970,128.132194,575.63,0,251000004608,260813504512,894195,1273,1.392014,0.348003
11,50000000,5000,6,no,yes,564.062247,564.512058,251000000112,ok,7000,444.631778,123.655129,549.81,0,251000004608,260813598720,602714,733,1.406462,0.234410
12,50000000,5000,8,no,yes,587.185998,587.654248,251000000112,ok,7000,427.121902,119.516231,510.48,0,251000004608,260813660160,451120,628,1.464120,0.183015
13,50000000,5000,10,no,yes,539.099562,539.529466,251000000112,ok,7000,465.220189,135.176851,628.87,0,251000004608,260813795328,365642,382,1.344219,0.134422


In [87]:
dfs_compression_on[4]

,keys,value_size,num_threads,rdbcompression,rdbchecksum,valkey_data_throughput_mb_s,actual_throughput_mb_s,rdb_file_size_bytes,status,port,save_duration_seconds,cpu_utilization_percent,cpu_total_time_seconds,io_read_bytes,io_write_bytes,memory_rss_bytes,context_switches_voluntary,context_switches_involuntary,speedup,efficiency
0,10000000,20000,1,yes,yes,207.124356,207.196792,200230000112,ok,7000,966.375968,99.956956,965.96,0,200230006784,207064252416,337,4813,1.000000,1.000000
1,10000000,20000,2,yes,yes,382.270704,382.404392,200230000112,ok,7000,523.608004,190.386700,996.88,0,200230006784,207064903680,169812,1515,1.845610,0.922805
2,10000000,20000,3,yes,yes,520.266871,520.448819,200230000112,ok,7000,384.725631,267.320375,1028.45,0,200230006784,207065133056,308628,1166,2.511858,0.837286
3,10000000,20000,4,yes,yes,654.342777,654.571615,200230000112,ok,7000,305.894719,336.759655,1030.13,0,200230006784,207065362432,338802,852,3.159178,0.789795
4,10000000,20000,6,yes,yes,737.896879,738.154937,200230000112,ok,7000,271.257415,387.569129,1051.31,0,200230006784,207065800704,432658,839,3.562579,0.593763
5,10000000,20000,8,yes,yes,708.350379,708.598104,200230000112,ok,7000,282.572023,374.743397,1058.92,0,200230006784,207066210304,310184,926,3.419928,0.427491
6,10000000,20000,10,yes,yes,699.757986,700.002706,200230000112,ok,7000,286.041752,375.329823,1073.60,0,200230006784,207066619904,251823,796,3.378444,0.337844


In [88]:
dfs_compression_off[4]

,keys,value_size,num_threads,rdbcompression,rdbchecksum,valkey_data_throughput_mb_s,actual_throughput_mb_s,rdb_file_size_bytes,status,port,save_duration_seconds,cpu_utilization_percent,cpu_total_time_seconds,io_read_bytes,io_write_bytes,memory_rss_bytes,context_switches_voluntary,context_switches_involuntary,speedup,efficiency
7,10000000,20000,1,no,yes,725.456018,725.709725,200230000112,ok,7000,275.909214,99.666842,274.99,0,200230006784,207065542656,20372,944,1.000000,1.000000
8,10000000,20000,2,no,yes,754.565192,754.829079,200230000112,ok,7000,265.265350,109.814569,291.30,0,200230006784,207065706496,1761431,807,1.040125,0.520063
9,10000000,20000,3,no,yes,754.696596,754.960529,200230000112,ok,7000,265.219164,104.223238,276.42,0,200230006784,207065788416,1127273,640,1.040306,0.346769
10,10000000,20000,4,no,yes,754.601218,754.865117,200230000112,ok,7000,265.252686,104.262846,276.56,0,200230006784,207065870336,808748,928,1.040175,0.260044
11,10000000,20000,6,no,yes,754.606232,754.870134,200230000112,ok,7000,265.250924,111.091036,294.67,0,200230006784,207066021888,514537,534,1.040182,0.173364
12,10000000,20000,8,no,yes,754.591098,754.854995,200230000112,ok,7000,265.256243,113.230134,300.35,0,200230006784,207066161152,375433,422,1.040161,0.130020
13,10000000,20000,10,no,yes,753.974156,754.237836,200230000112,ok,7000,265.473290,122.969056,326.45,0,200230006784,207066312704,297200,703,1.039311,0.103931


In [89]:
dfs_compression_on[5]


,keys,value_size,num_threads,rdbcompression,rdbchecksum,valkey_data_throughput_mb_s,actual_throughput_mb_s,rdb_file_size_bytes,status,port,save_duration_seconds,cpu_utilization_percent,cpu_total_time_seconds,io_read_bytes,io_write_bytes,memory_rss_bytes,context_switches_voluntary,context_switches_involuntary,speedup,efficiency
0,5000000,64000,1,yes,yes,151.953349,151.969965,320115000112,ok,7000,2106.435964,99.959364,2105.58,0,320115007488,329260044288,882,9727,1.000000,1.000000
1,5000000,64000,2,yes,yes,288.214492,288.246007,320115000112,ok,7000,1110.561784,192.523282,2138.09,0,320115007488,329260785664,211317,3372,1.896730,0.948365
2,5000000,64000,3,yes,yes,412.108641,412.153704,320115000112,ok,7000,776.688398,277.041605,2151.75,0,320115007488,329261109248,277323,1892,2.712073,0.904024
3,5000000,64000,4,yes,yes,523.520816,523.578062,320115000112,ok,7000,611.398802,351.466178,2148.86,0,320115007488,329261498368,292314,1503,3.445273,0.861318
4,5000000,64000,6,yes,yes,660.413747,660.485962,320115000112,ok,7000,484.665865,454.630738,2203.44,0,320115007488,329262280704,379640,1436,4.346161,0.724360
5,5000000,64000,8,yes,yes,690.667899,690.743422,320115000112,ok,7000,463.435466,489.021701,2266.30,0,320115007488,329262727168,331837,1365,4.545263,0.568158
6,5000000,64000,10,yes,yes,670.385171,670.458477,320115000112,ok,7000,477.456862,463.845046,2214.66,0,320115007488,329263521792,270511,1391,4.411783,0.441178


In [90]:
dfs_compression_off[5]


,keys,value_size,num_threads,rdbcompression,rdbchecksum,valkey_data_throughput_mb_s,actual_throughput_mb_s,rdb_file_size_bytes,status,port,save_duration_seconds,cpu_utilization_percent,cpu_total_time_seconds,io_read_bytes,io_write_bytes,memory_rss_bytes,context_switches_voluntary,context_switches_involuntary,speedup,efficiency
7,5000000,64000,1,no,yes,614.648875,614.716086,320115000112,ok,7000,520.752600,98.903011,515.04,0,320115007488,329261285376,182970,2679,1.000000,1.000000
8,5000000,64000,2,no,yes,742.701642,742.782855,320115000112,ok,7000,430.967137,123.215892,531.02,0,320115007488,329261555712,1831845,2313,1.208335,0.604167
9,5000000,64000,3,no,yes,754.768815,754.851347,320115000112,ok,7000,424.076874,101.212781,429.22,0,320115007488,329261694976,1382511,1683,1.227967,0.409322
10,5000000,64000,4,no,yes,754.778513,754.861047,320115000112,ok,7000,424.071425,102.216272,433.47,0,320115007488,329261834240,1024474,1639,1.227983,0.306996
11,5000000,64000,6,no,yes,754.740981,754.823510,320115000112,ok,7000,424.092514,103.133157,437.38,0,320115007488,329262100480,691660,980,1.227922,0.204654
12,5000000,64000,8,no,yes,754.758872,754.841404,320115000112,ok,7000,424.082461,107.304603,455.06,0,320115007488,329262354432,501642,527,1.227951,0.153494
13,5000000,64000,10,no,yes,754.764390,754.846922,320115000112,ok,7000,424.079361,105.989596,449.48,0,320115007488,329262620672,387628,447,1.227960,0.122796
